## Analyze whether SNWD varies more from year to year or from place to place.

In [2]:
import pandas as pd
import numpy as np
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
#file_index='BBBSBBBB'
file_index='BBSBSBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
953
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
| 2840.872029540231|-1025.2231097807962|-345.8072054342014|    412.1|BBSBSBSB| 47.3864| -92.8389|       SNWD|0.23536631597571686| 0.1357829369609797|0.12445324347418109|0.27092159

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [13]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USW00094931,1989.0,-345.807205
1,USC00214652,1996.0,438.246712
2,USW00014918,1989.0,-739.568878
3,USC00218543,1975.0,880.716217
4,USC00218543,1969.0,-1719.791201


In [14]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,CA006020559,CA006025203,CA006025205,CA006032119,CA006034075,CA006036904,USC00210059,USC00210515,USC00210520,USC00210643,...,USC00217460,USC00218543,USC00218618,USC00218621,USC00218700,USC00218763,USC00219059,USC00219173,USW00014918,USW00094931
year,,,,,,,,,,,,,,,,,,,,,
1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.342705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-488.162515,NaN
1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.117751,...,NaN,NaN,-106.823583,NaN,NaN,362.964820,1401.421696,NaN,544.048738,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-416.532177,...,373.986626,NaN,-526.919299,NaN,NaN,NaN,405.514701,NaN,-72.734605,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213.305221,NaN,-427.997304,...,288.718552,NaN,1385.280481,NaN,NaN,2588.760802,871.787517,NaN,656.296019,NaN
1952.0,NaN,NaN,NaN,NaN,NaN,NaN,162.142134,NaN,NaN,-716.366286,...,131.364964,NaN,-879.850975,NaN,NaN,540.678260,282.935115,NaN,110.432238,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,179.483597,-97.943385,NaN,-311.549318,...,-471.610221,NaN,-169.896366,NaN,NaN,-297.761481,-430.166032,NaN,-679.560027,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,-40.033294,-253.171967,NaN,-375.909564,...,-718.657727,NaN,-863.185106,NaN,NaN,202.774290,-403.573471,NaN,-751.439447,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,1198.783749,1078.132460,NaN,1802.767111,...,139.503127,NaN,774.711927,NaN,NaN,1142.508309,875.590400,NaN,741.808338,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,-83.450469,81.619539,NaN,266.180872,...,-663.437042,NaN,139.571605,NaN,NaN,956.755894,467.669179,NaN,488.996067,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [15]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  710.360614167
RMS removing mean-by-station=  672.647841391
RMS removing mean-by-year   =  457.688315369


In [16]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 710.360614167
0 after removing mean by year    = 457.688315369
0 after removing mean by stations= 415.401463928
1 after removing mean by year    = 413.847657634
1 after removing mean by stations= 413.664220048
2 after removing mean by year    = 413.626305238
2 after removing mean by stations= 413.615185778
3 after removing mean by year    = 413.611043369
3 after removing mean by stations= 413.609226055
4 after removing mean by year    = 413.608341153
4 after removing mean by stations= 413.607883279
